In [8]:
import pandas as pd
import pickle

df = pd.read_csv('big_data\\troop_movements_1m.csv')

with open('data\\model.pkl', 'rb') as f:
    model = pickle.load(f)

In [9]:
# Clean up the data.

# df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

df.columns = df.columns.str.strip()

df['unit_type'].replace('invalid_unit', 'unknown', inplace=True)



df[['location_x', 'location_y']].ffill()

print("test in valid unit: ")
test1 = df[df['unit_type'] == 'invalid_unit']
print(test1)

print("test: ")
test = df[df['location_x'] == None]
print(test)

print(df)


C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_1052\710341891.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['unit_type'].replace('invalid_unit', 'unknown', inplace=True)


test in valid unit: 
Empty DataFrame
Columns: [timestamp, unit_id, unit_type, location_x, location_y, destination_x, destination_y, homeworld]
Index: []
test: 
Empty DataFrame
Columns: [timestamp, unit_id, unit_type, location_x, location_y, destination_x, destination_y, homeworld]
Index: []
                  timestamp  unit_id     unit_type  location_x  location_y  \
0       2023-06-03 03:19:15   919214  tie_silencer         2.0         5.0   
1       2023-02-24 13:50:40  9467154  stormtrooper         9.0         0.0   
2       2023-03-29 19:54:55  6585778  tie_silencer         0.0         6.0   
3       2023-04-30 00:58:11  3878023  tie_silencer         4.0         2.0   
4       2023-04-10 22:00:26  5537117         at-st         6.0         8.0   
...                     ...      ...           ...         ...         ...   
999995  2023-05-03 18:48:56  3553240   tie_fighter         8.0         4.0   
999996  2023-05-25 13:45:35  1662636         at-at         0.0         3.0   
999997

In [10]:
# Save the data to Parquet.
! pip install pyarrow
! pip install fastparquet

import pyarrow
import fastparquet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
# Load the parquet file

df.to_parquet('big_data\\troop_movements_1m.parquet')


In [22]:
# Add predictions to DF.
X_encoded = pd.get_dummies(df[['homeworld', 'unit_type']])

predictions = model.predict(X_encoded)
print(f"Predictions: {predictions}")

df['predictions (resistance)'] = predictions[:, 1]
print(df)

Predictions: [[ True False]
 [False False]
 [ True False]
 ...
 [False  True]
 [False  True]
 [ True False]]
                  timestamp  unit_id     unit_type  location_x  location_y  \
0       2023-06-03 03:19:15   919214  tie_silencer         2.0         5.0   
1       2023-02-24 13:50:40  9467154  stormtrooper         9.0         0.0   
2       2023-03-29 19:54:55  6585778  tie_silencer         0.0         6.0   
3       2023-04-30 00:58:11  3878023  tie_silencer         4.0         2.0   
4       2023-04-10 22:00:26  5537117         at-st         6.0         8.0   
...                     ...      ...           ...         ...         ...   
999995  2023-05-03 18:48:56  3553240   tie_fighter         8.0         4.0   
999996  2023-05-25 13:45:35  1662636         at-at         0.0         3.0   
999997  2023-05-17 12:16:37  2362262   tie_fighter         6.0         8.0   
999998  2023-03-17 08:04:26  7669749   tie_fighter         9.0         3.0   
999999  2023-05-19 12:29:02  2187